In [1]:
import pandas as pd
import pandas_datareader.data as web
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor
import requests
import time
import random

### Get info about S&P500 companies and get their tickers

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36"
}

response = requests.get(url, headers=headers)
response.raise_for_status()

tables = pd.read_html(response.text)
df_sp500 = tables[0]
df_sp500.sample(20)

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
470,VZ,Verizon,Communication Services,Integrated Telecommunication Services,"New York City, New York",1983-11-30,732712,1983 (1877)
146,DVN,Devon Energy,Energy,Oil & Gas Exploration & Production,"Oklahoma City, Oklahoma",2000-08-30,1090012,1971
87,CARR,Carrier Global,Industrials,Building Products,"Palm Beach Gardens, Florida",2020-04-03,1783180,"2020 (1915, United Technologies spinoff)"
32,AMGN,Amgen,Health Care,Biotechnology,"Thousand Oaks, California",1992-01-02,318154,1980
144,DELL,Dell Technologies,Information Technology,"Technology Hardware, Storage & Peripherals","Round Rock, Texas",2024-09-23,1571996,2016
399,ROL,"Rollins, Inc.",Industrials,Environmental & Facilities Services,"Atlanta, Georgia",2018-10-01,84839,1948
412,SPG,Simon Property Group,Real Estate,Retail REITs,"Indianapolis, Indiana",2002-06-26,1063761,2003
285,LEN,Lennar,Consumer Discretionary,Homebuilding,"Miami, Florida",2005-10-04,920760,1954
334,NKE,"Nike, Inc.",Consumer Discretionary,"Apparel, Accessories & Luxury Goods","Washington County, Oregon",1988-11-30,320187,1964
456,UBER,Uber,Industrials,Passenger Ground Transportation,"San Francisco, California",2023-12-18,1543151,2009


In [3]:
df_sp500.to_csv("stock_data/sp500_tickers_info.csv", index=False)

In [4]:
# get tickers
tickers = df_sp500["Symbol"].tolist()
print(f"Fetched {len(tickers)} tickers!")
tickers = [ticker.replace('.', '-') + '.US' for ticker in tickers]
start = datetime(2020, 1, 1)
end = datetime.today()

Fetched 502 tickers!


### Use stooq to get the stock price data

In [5]:
tickers_errors = []

In [6]:
def fetch_ticker_data(ticker):
    """Downloads data with a polite delay and jitter."""
    # Adding jitter: sleep between 3 and 7 seconds
    # This prevents 'bursty' traffic where all threads wake up at once
    delay = 5 + random.uniform(-2, 2)
    print(f"Waiting {delay:.2f}s before fetching {ticker}...")
    time.sleep(delay)
    url = f"https://stooq.com/q/d/l/?s={ticker}&d1={start:%Y%m%d}&d2={end:%Y%m%d}&i=d"
    try:
        df = pd.read_csv(url, parse_dates=["Date"])
        if df.empty:
            print(f"Warning: No data for {ticker}")
            return None
        df["Ticker"] = ticker.split('.')[0].upper() 
        return df
    except Exception as e:
        print(f"Error fetching {ticker}: {e}")
        tickers_errors.append(ticker)
        return None

In [7]:
with ThreadPoolExecutor(max_workers=15) as executor:
    results = list(executor.map(fetch_ticker_data, tickers))

dfs = [df for df in results if df is not None]

Waiting 3.61s before fetching MMM.US...
Waiting 6.40s before fetching AOS.US...
Waiting 4.78s before fetching ABT.US...
Waiting 4.32s before fetching ABBV.US...
Waiting 6.85s before fetching ACN.US...
Waiting 3.48s before fetching ADBE.US...
Waiting 6.55s before fetching AMD.US...
Waiting 5.74s before fetching AES.US...
Waiting 6.86s before fetching AFL.US...
Waiting 4.19s before fetching A.US...
Waiting 6.25s before fetching APD.US...
Waiting 5.02s before fetching ABNB.US...
Waiting 4.48s before fetching AKAM.US...
Waiting 6.18s before fetching ALB.US...
Waiting 3.53s before fetching ARE.US...
Waiting 5.56s before fetching ALGN.US...Waiting 3.61s before fetching ALLE.US...

Waiting 3.66s before fetching LNT.US...
Waiting 3.29s before fetching ALL.US...
Waiting 3.12s before fetching GOOGL.US...
Waiting 4.18s before fetching GOOG.US...
Waiting 5.98s before fetching MO.US...
Waiting 4.40s before fetching AMZN.US...
Waiting 3.50s before fetching AMCR.US...
Waiting 3.57s before fetching AE

In [8]:
tickers_errors

['Q.US', 'LIN.US']

In [9]:
with ThreadPoolExecutor(max_workers=15) as executor:
    results2 = list(executor.map(fetch_ticker_data, tickers_errors))

Waiting 5.18s before fetching Q.US...
Waiting 5.68s before fetching LIN.US...
Error fetching Q.US: Missing column provided to 'parse_dates': 'Date'


In [10]:
final_df = (pd.concat(dfs, ignore_index=True)
                .set_index(["Ticker", "Date"])
                .sort_index())
final_df = final_df.reset_index()
final_df

,Ticker,Date,Open,High,Low,Close,Volume
0,A,2020-01-02,84.3205,84.7649,83.6339,84.3693,1.418583e+06
1,A,2020-01-03,83.1137,83.7614,82.9482,83.0150,1.139251e+06
2,A,2020-01-06,82.4570,83.2632,82.0634,83.2632,2.030496e+06
3,A,2020-01-07,82.4182,83.6937,82.3982,83.5163,1.755393e+06
4,A,2020-01-08,84.3793,84.8825,83.6339,84.3404,1.882163e+06
...,...,...,...,...,...,...,...
754986,ZTS,2026-01-29,123.1200,123.3400,121.3300,121.7200,4.729118e+06
754987,ZTS,2026-01-30,121.1600,124.8700,120.0600,124.8200,5.247316e+06
754988,ZTS,2026-02-02,124.0800,125.1200,123.0400,124.9400,4.109062e+06
754989,ZTS,2026-02-03,123.6750,126.3900,122.2100,122.4100,3.816195e+06


In [11]:
final_df.Ticker.value_counts()

A       1531
NTRS    1531
NSC     1531
NRG     1531
NOW     1531
        ... 
VLTO     587
SOLV     463
GEV      462
SW       395
SNDK     245
Name: Ticker, Length: 500, dtype: int64

In [12]:
final_df.to_csv("stock_data/sp500_stock_prices_v2.csv", index=False)